In [1]:
!pip install nflows torch matplotlib numpy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from scipy.stats import ks_2samp, wasserstein_distance, skew, kurtosis
from nflows.distributions import StandardNormal
from nflows.transforms import CompositeTransform, MaskedAffineAutoregressiveTransform
from nflows.flows import Flow

# === Reproducibility
torch.manual_seed(42)
np.random.seed(42)

# === Load price series
df = pd.read_csv("raw (FX + EQ).csv")
usdzar_prices = df["USDZAR"].values
usdzar_returns = np.log(usdzar_prices[1:] / usdzar_prices[:-1])

# === Discover residual files
residuals_dir = "residuals_by_model"
residual_files = glob(os.path.join(residuals_dir, "*", "*.csv"))
print(f"Found {len(residual_files)} residual files.")

def train_nf_on_residuals(file_path, model_key, output_dir="nf_models", epochs=100): #dev epoch
    residuals = pd.read_csv(file_path).values.astype(np.float32)
    residuals = residuals[~np.isnan(residuals)].flatten().reshape(-1, 1)
    residuals_tensor = torch.tensor(residuals, dtype=torch.float32)
    dataset = TensorDataset(residuals_tensor)
    loader = DataLoader(dataset, batch_size=128, shuffle=True)

    transform = CompositeTransform([
        MaskedAffineAutoregressiveTransform(features=1, hidden_features=16)
        for _ in range(5)
    ])
    base_dist = StandardNormal([1])
    flow = Flow(transform, base_dist)

    optimizer = torch.optim.Adam(flow.parameters(), lr=1e-3)
    loss_history = []

    for epoch in range(epochs):
        total_loss = 0.0
        for batch in loader:
            x = batch[0]
            loss = -flow.log_prob(x).mean()
            if torch.isnan(loss):
                print(f"❌ NaN loss encountered at epoch {epoch+1} for {model_key}")
                return None, [], residuals
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        loss_history.append(total_loss / len(loader))
        if (epoch + 1) % 100 == 0 or epoch == 0:
            print(f"[{model_key}] Epoch {epoch+1}: Loss = {loss_history[-1]:.4f}")

    # Save model and loss
    model_subdir = os.path.join(output_dir, model_key)
    os.makedirs(model_subdir, exist_ok=True)
    torch.save(flow.state_dict(), os.path.join(model_subdir, "nf.pth"))
    np.savetxt(os.path.join(model_subdir, "loss_history.txt"), loss_history)

    plt.plot(loss_history)
    plt.title(f"NF Training Loss - {model_key}")
    plt.xlabel("Epoch")
    plt.ylabel("NLL")
    plt.grid()
    os.makedirs("plots", exist_ok=True)
    plt.savefig(f"plots/{model_key}_loss.png")
    plt.close()

    return flow, loss_history, residuals



# === Train and evaluate on all residual files ===
trained_flows = {}
trained_residuals = {}

for file_path in residual_files:
    asset = os.path.basename(os.path.dirname(file_path))        # fx_USDZAR
    model = os.path.splitext(os.path.basename(file_path))[0]    # eGARCH
    model_key = f"{model}_{asset}_residuals_synthetic"
    print(f"\n=== Training NF for {model_key} ===")

    flow, loss_history, resid_array = train_nf_on_residuals(file_path, model_key)
    if flow is None:
        continue

    trained_flows[model_key] = flow
    trained_residuals[model_key] = resid_array

    with torch.no_grad():
        synthetic_resid = flow.sample(len(resid_array)).detach().numpy().flatten()

    # Save synthetic residuals
    synthetic_dir = "nf_generated_residuals"
    os.makedirs(synthetic_dir, exist_ok=True)
    synthetic_path = os.path.join(synthetic_dir, f"{model_key}_synthetic.csv")
    pd.DataFrame({"residual": synthetic_resid}).to_csv(synthetic_path, index=False)
    print(f"✅ Saved synthetic residuals to: {synthetic_path}")




Found 120 residual files.

=== Training NF for equity_AMZN_Chrono_Split_residuals_eGARCH_residuals_synthetic ===
[equity_AMZN_Chrono_Split_residuals_eGARCH_residuals_synthetic] Epoch 1: Loss = 1.5622
[equity_AMZN_Chrono_Split_residuals_eGARCH_residuals_synthetic] Epoch 100: Loss = 1.5196
✅ Saved synthetic residuals to: nf_generated_residuals\equity_AMZN_Chrono_Split_residuals_eGARCH_residuals_synthetic_synthetic.csv

=== Training NF for equity_AMZN_TS_CV_residuals_eGARCH_residuals_synthetic ===
[equity_AMZN_TS_CV_residuals_eGARCH_residuals_synthetic] Epoch 1: Loss = 1.4822
[equity_AMZN_TS_CV_residuals_eGARCH_residuals_synthetic] Epoch 100: Loss = 1.4584
✅ Saved synthetic residuals to: nf_generated_residuals\equity_AMZN_TS_CV_residuals_eGARCH_residuals_synthetic_synthetic.csv

=== Training NF for equity_CAT_Chrono_Split_residuals_eGARCH_residuals_synthetic ===
[equity_CAT_Chrono_Split_residuals_eGARCH_residuals_synthetic] Epoch 1: Loss = 1.7117
[equity_CAT_Chrono_Split_residuals_eGARCH_

In [3]:
def find_matching_model_key(trained_dict, asset, model_config):
    for key in trained_dict.keys():
        if asset in key and model_config in key:
            return key
    raise KeyError(f"No model found for asset={asset}, model={model_config}")

print("\nAvailable model keys:")
for k in trained_flows.keys():
    print("-", k)

def evaluate_synthetic_residuals(asset, model_config, trained_flows, trained_residuals, real_prices, real_returns):
    model_key = find_matching_model_key(trained_flows, asset, model_config)
    flow = trained_flows[model_key]
    residuals = trained_residuals[model_key].flatten()

    with torch.no_grad():
        synthetic_residuals = flow.sample(len(residuals)).detach().numpy().flatten()

    # Residual comparison
    plt.hist(residuals, bins=60, alpha=0.6, label="Original")
    plt.hist(synthetic_residuals, bins=60, alpha=0.6, label="Synthetic")
    plt.legend()
    plt.title(f"Residual Distribution: {model_key}")
    plt.savefig(f"plots/{model_key}_residual_comparison.png")
    plt.close()

    # Price simulation
    synthetic_returns = synthetic_residuals * np.std(real_returns)
    synthetic_prices = real_prices[0] * np.cumprod(np.exp(synthetic_returns))
    synthetic_prices = synthetic_prices[:len(real_prices)]

    plt.figure(figsize=(10, 5))
    plt.plot(real_prices, label="Real USDZAR")
    plt.plot(synthetic_prices, label="Synthetic NF", linestyle="--")
    plt.legend()
    plt.title(f"{asset} - Real vs Synthetic Price Path")
    plt.savefig(f"plots/{model_key}_price_comparison.png")
    plt.close()

    # Evaluation metrics
    metrics = {
        "KS Distance": ks_2samp(real_returns, synthetic_returns).statistic,
        "Wasserstein Distance": wasserstein_distance(real_returns, synthetic_returns),
        "Skewness (Real)": skew(real_returns),
        "Skewness (Synthetic)": skew(synthetic_returns),
        "Kurtosis (Real)": kurtosis(real_returns),
        "Kurtosis (Synthetic)": kurtosis(synthetic_returns)
    }

    print(f"\n📊 Metrics for {model_key}:")
    for k, v in metrics.items():
        print(f"{k}: {v:.5f}")

    return synthetic_returns

# === Example usage: USDZAR, eGARCH
evaluate_synthetic_residuals("USDZAR", "eGARCH", trained_flows, trained_residuals, usdzar_prices, usdzar_returns)


Available model keys:
- equity_AMZN_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_AMZN_TS_CV_residuals_eGARCH_residuals_synthetic
- equity_CAT_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_CAT_TS_CV_residuals_eGARCH_residuals_synthetic
- equity_MSFT_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_MSFT_TS_CV_residuals_eGARCH_residuals_synthetic
- equity_NVDA_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_NVDA_TS_CV_residuals_eGARCH_residuals_synthetic
- equity_PG_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_PG_TS_CV_residuals_eGARCH_residuals_synthetic
- equity_WMT_Chrono_Split_residuals_eGARCH_residuals_synthetic
- equity_WMT_TS_CV_residuals_eGARCH_residuals_synthetic
- fx_EURUSD_Chrono_Split_residuals_eGARCH_residuals_synthetic
- fx_EURUSD_TS_CV_residuals_eGARCH_residuals_synthetic
- fx_EURZAR_Chrono_Split_residuals_eGARCH_residuals_synthetic
- fx_EURZAR_TS_CV_residuals_eGARCH_residuals_synthetic
- fx_GBPCNY_Chrono_Split_

array([0.0111035 , 0.00936287, 0.01664994, ..., 0.00743666, 0.00613614,
       0.01649721], dtype=float32)